In [1]:
import json
import os
from imageai.Prediction.Custom import CustomImagePrediction
import random
import shutil
from pprint import pprint


In [2]:
with open('pairing.json') as json_file:  
    data = json.load(json_file)

bad_keys = []
def clean(name: str) -> str:
    return name.replace(' ', '').replace('_', '').replace('/', '')
for key in data:
    if '?' in data[key]:
        bad_keys.append(key)
    data[key] = clean(data[key])

for key in bad_keys:
    del data[key]


    
    

In [3]:
data['audi_s3'] = '아우디S3S3(96년~현재)'
data['bmw_5series'] = 'BMW5시리즈5시리즈(F10)(10~16년)'
# data['lexus_rs'] = ''
data['volvo_s60'] = '볼보S60S60(00년~현재)'
data['KakaoTalk_Video_2019-06-04-02-18-11'] = '벤츠E-클래스W212(09~17년)'
data['KakaoTalk_Video_2019-06-04-02-17-55'] = 'BMW3시리즈3시리즈(F30)(12~18년)'
data['KakaoTalk_Video_2019-06-04-11-40-03'] = '벤츠C-클래스W204(07~14년)'
data['KakaoTalk_Video_2019-06-04-11-43-12'] = '르노삼성클리오(18년~현재)'
data['KakaoTalk_Video_2019-06-04-11-43-42'] = '벤츠CLS-클래스C257(18년~현재)'
data['KakaoTalk_Video_2019-06-04-11-44-35'] = '기아카니발뉴카니발(06~10년)'
data['KakaoTalk_Video_2019-06-04-11-45-10'] = '기아카니발뉴카니발(06~10년)'
data['KakaoTalk_Video_2019-06-04-11-45-30'] = '현대그랜저그랜저HG(11~16년)'
data['KakaoTalk_Video_2019-06-04-11-46-01'] = '현대그랜저그랜저HG(11~16년)'


In [4]:
root = '/Volumes/TriveStorage/code/trive-image-recognition/complete_manual/phone_videos'
mapped_data = {}
total = 0
for file_name in [x for x in os.listdir(root) if x != '.DS_Store']:
    if file_name not in data:
        continue
    mapped_data[data[file_name]] = [os.path.join(root, file_name, x) for x in os.listdir(os.path.join(root, file_name)) if x != '.DS_Store']



In [5]:
exec_path = '/Volumes/TriveStorage/code/trive-image-recognition'

p = CustomImagePrediction()
p.setModelTypeAsResNet()
path_to_resnet = 'vehicle_direction.h5'
path_to_models = 'vehicle_direction.json'
p.setModelPath(path_to_resnet)
p.setJsonPath(path_to_models)
p.loadModel(num_objects=5)



Instructions for updating:
Colocations handled automatically by placer.


In [6]:
combined_path_direction =  os.path.join(exec_path, 'complete_manual', 'directional_cleaned')
directions = ['back', 'front', 'mix', 'side', 'trash']


In [7]:
for key in mapped_data:
    dst_folder = os.path.join(combined_path_direction, key)
    if not os.path.exists(dst_folder):
        os.mkdir(dst_folder)
    for direction in directions:
        dir_path = os.path.join(dst_folder, direction)
        if not os.path.exists(dir_path):
            os.mkdir(dir_path)
    for img in mapped_data[key]:
        basename = os.path.basename(img)
        if os.path.exists(os.path.join(dst_folder, 'back', basename)):
            continue
        if os.path.exists(os.path.join(dst_folder, 'front', basename)):
            continue
        if os.path.exists(os.path.join(dst_folder, 'mix', basename)):
            continue
        if os.path.exists(os.path.join(dst_folder, 'side', basename)):
            continue
        if os.path.exists(os.path.join(dst_folder, 'trash', basename)):
            continue
        try:
            predictions, probabilities = p.predictImage(img, result_count=1)
            predicted_direction = predictions[0]
            direction_path = os.path.join(dst_folder, predicted_direction)
            shutil.copy2(img, direction_path)
        except Exception as e:
            print(e)

